# Skin Disease Detection using Mobile Application
## Final Year Project 2
Ahmad Daniel Ikhwan Bin Rosli <br>
1201103071

In [5]:
import os
import shutil
import random
from pathlib import Path
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split

In [1]:
img_size = (224, 224)
split_ratio = [0.8, 0.1, 0.1]  
random_seed = 42

In [2]:
!kaggle datasets download -d shubhamgoel27/dermnet --unzip -p datasets/dermnet_raw

Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)


In [ ]:
target_classes = {
    "Acne and Rosacea Photos"
    "Eczema Photos",
    "Psoriasis pictures Lichen Planus and related diseases",
    "Tinea Ringworm Candidiasis and other Fungal Infections",
    "Melanoma Skin Cancer Nevi and Moles"
}

raw_path = Path("datasets/dermnet_raw")     
prep_path = Path("datasets/dermnet_prep")    

for split in ['train', 'test']:
    folder = raw_path / split
    for sub in folder.iterdir():
        if sub.is_dir() and sub.name not in target_classes:
            print(f"Deleting {sub}")
            shutil.rmtree(sub)